In [79]:
from keras.preprocessing.text import Tokenizer

In [80]:
from servier.data import get_data

In [81]:
df = get_data()

In [82]:
docs = df.smiles.values

In [83]:
t = Tokenizer(char_level=True)
t.fit_on_texts(docs)

encoded_docs = t.texts_to_sequences(docs)#, mode='count')

In [86]:
print(len(docs[0]), len(encoded_docs[0]))

47 47


In [90]:
print(list(docs[0]))
encoded_docs[0]

['C', 'c', '1', 'c', 'c', 'c', 'c', '(', 'N', '2', 'C', 'C', 'N', '(', 'C', '(', '=', 'O', ')', 'C', '3', '4', 'C', 'C', '5', 'C', 'C', '(', 'C', 'C', '(', 'C', '5', ')', 'C', '3', ')', 'C', '4', ')', 'C', 'C', '2', ')', 'c', '1', 'C']


[1,
 1,
 5,
 1,
 1,
 1,
 1,
 2,
 4,
 7,
 1,
 1,
 4,
 2,
 1,
 2,
 8,
 6,
 3,
 1,
 9,
 12,
 1,
 1,
 24,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 24,
 3,
 1,
 9,
 3,
 1,
 12,
 3,
 1,
 1,
 7,
 3,
 1,
 5,
 1]

# Sandbox

In [91]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [99]:
cv = CountVectorizer(analyzer='char', lowercase=False)
tfidf = TfidfVectorizer(analyzer='char', lowercase=False)
hash_ = HashingVectorizer()

In [100]:
X = hash_.fit_transform(df.smiles.values)
X

<4999x1048576 sparse matrix of type '<class 'numpy.float64'>'
	with 26030 stored elements in Compressed Sparse Row format>

In [114]:
len(sigles)

29

# Pipeline

In [110]:
from keras.layers.embeddings import Embedding
sigles = {l for word in df.smiles.values for l in word}
max_ = max([len(w) for w in df.smiles.values])

In [139]:
from keras.layers import BatchNormalization
from keras.layers.core import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, LSTM, GRU, Bidirectional, TimeDistributed, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D, MaxPooling1D

embedding_vector_length, max_length, vocab_size = 32, 74, 27

def get_model(size, verbose=False):
    model = Sequential([
        Dense(size, input_shape=(size,), activation="relu"),
        Dense(256, activation="sigmoid"),
        Dense(64, activation="sigmoid"),
        Dense(34, activation="sigmoid"),
        Dense(16, activation="sigmoid"),
        BatchNormalization(axis=1),
        Dense(2, activation="softmax")
    ])
    if verbose:
        model.summary()
    model.compile(optimizer=Adam(lr=0.00001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

def GRU2_model(X,Y, dropout=0, epochs=10, batch_size=32):
    """
    Module that creates a 2 layer GRU for property prediction from SMILES
    """

    model = Sequential()
    model.add(Embedding(max_length, embedding_vector_length, input_length=max_length))

    model.add(GRU(100, return_sequences=True, dropout=dropout))
    model.add(GRU(100, dropout=dropout))

    model.add(Dense(2))

    model.compile(optimizer=Adam(lr=0.00001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    #model.summary()
    history = model.fit(X, Y, validation_split=0.1, epochs=epochs, batch_size=batch_size, shuffle=True, verbose=1)
    save_model("gru_2layer", model, history, dropout, epochs, batch_size)
    return model

def GRU_model(X,Y, dropout=0, epochs=10, batch_size=32):

    """
    Module that creates a 1 layer GRU for property prediction from SMILES
    """
    
    vectorize_layer = TextVectorization(
       max_tokens=vocab_size,
       output_mode='int',
       output_sequence_length=max_length
    )
    model = Sequential()
    model.add(vectorize_layer)
    #model.add(Embedding(max_length, embedding_vector_length, input_length=max_length))

    model.add(GRU(100, dropout=dropout))

    model.add(Dense(2, activation="softmax"))

    model.compile(optimizer=Adam(lr=0.00001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    #model.summary()
    history = model.fit(X, Y, validation_split=0.1, epochs=epochs, batch_size=batch_size, shuffle=True, verbose=1)
    #save_model("gru_1layer", model, history, dropout, epochs, batch_size)
    return model

In [140]:
from servier.data import get_data, get_X_y
df = get_data()
X, y = get_X_y(df)

In [141]:
X.head()


,mol_id,smiles
0,CID2999678,Cc1cccc(N2CCN(C(=O)C34CC5CC(CC(C5)C3)C4)CC2)c1C
1,CID2999679,Cn1ccnc1SCC(=O)Nc1ccc(Oc2ccccc2)cc1
2,CID2999672,COc1cc2c(cc1NC(=O)CN1C(=O)NC3(CCc4ccccc43)C1=O...
3,CID5390002,O=C1/C(=C/NC2CCS(=O)(=O)C2)c2ccccc2C(=O)N1c1cc...
4,CID2999670,NC(=O)NC(Cc1ccccc1)C(=O)O


In [142]:
#morgan_fp_encoder = make_pipeline(MorganFingerprintEncoder(size=self.fp_size))
model = GRU_model(X.smiles.values, y)

Epoch 1/10


ValueError: in user code:

    /Users/jeanbizot/miniconda3/envs/servier/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Users/jeanbizot/miniconda3/envs/servier/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/jeanbizot/miniconda3/envs/servier/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/jeanbizot/miniconda3/envs/servier/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/jeanbizot/miniconda3/envs/servier/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        retur

In [ ]:
y_pred = model.predict_classes(X_val, batch_size=1, verbose=1)